In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.models import load_model
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [3]:
def load_data(folder_path, num_images=None):
    images = []

    for filename in os.listdir(folder_path):
        if num_images is not None and len(images) >= num_images:
            break

        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64, 64))  
        images.append(img)

    return np.array(images)

In [4]:
def build_and_train_autoencoder(X_train, X_test):
    model = models.Sequential([
        layers.Input(shape=(64, 64, 3)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2), padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')  # Output with 3 channels for color images
    ])

    model.compile(optimizer='adam', loss='mse')  # Using mean squared error for color images

    model.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))
    
    return model

In [5]:
folder_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba"
X = load_data(folder_path, num_images=100000)
print(len(X))

100000


In [6]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(-1, 64, 64, 3)  
X_test = X_test.reshape(-1, 64, 64, 3)

In [7]:
autoencoder_model = build_and_train_autoencoder(X_train, X_test)
autoencoder_model.save("saved_models/autoencoder_model.h5")

Epoch 1/10
2500/2500 [==============================] - 393s 157ms/step - loss: 0.0122 - val_loss: 0.0081
Epoch 2/10
2500/2500 [==============================] - 370s 148ms/step - loss: 0.0075 - val_loss: 0.0067
Epoch 3/10
2500/2500 [==============================] - 375s 150ms/step - loss: 0.0065 - val_loss: 0.0062
Epoch 4/10
2500/2500 [==============================] - 370s 148ms/step - loss: 0.0059 - val_loss: 0.0056
Epoch 5/10
2500/2500 [==============================] - 371s 148ms/step - loss: 0.0055 - val_loss: 0.0055
Epoch 6/10
2500/2500 [==============================] - 372s 149ms/step - loss: 0.0052 - val_loss: 0.0052
Epoch 7/10
2500/2500 [==============================] - 370s 148ms/step - loss: 0.0050 - val_loss: 0.0048
Epoch 8/10
2500/2500 [==============================] - 370s 148ms/step - loss: 0.0048 - val_loss: 0.0047
Epoch 9/10
2500/2500 [==============================] - 1408s 563ms/step - loss: 0.0046 - val_loss: 0.0045
Epoch 10/10
2500/2500 [======================

/Users/vladcalomfirescu/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
def classify_large_image(autoencoder_model, larger_image_path, threshold=0.5):
    larger_image = cv2.imread(larger_image_path)
    larger_image_resized = cv2.resize(larger_image, (64, 64)) / 255.0
    larger_image_input = np.expand_dims(larger_image_resized, axis=0)

    # Assuming autoencoder_model output has shape (batch_size, 64, 64, 3)
    prediction = autoencoder_model.predict(larger_image_input)

    # You can use a simple threshold or a classifier based on your needs
    if np.mean(prediction) > threshold:
        print("Face detected!")
    else:
        print("No face detected.")

autoencoder_model = load_model("saved_models/autoencoder_model.h5")
larger_image_path = "/Users/vladcalomfirescu/Desktop/MyFiles/FAC/master/an 1/sem 1/PPC/BlurFacesML/data/img_align_celeba/201923.jpg"
classify_large_image(autoencoder_model, larger_image_path, threshold=0.9)


1/1 [==============================] - 0s 51ms/step
No face detected.
